In [64]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains import LLMChain
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.exceptions import OutputParserException
from pydantic import BaseModel, Field
import json
import time

In [65]:
with open('system_prompt.txt', 'r') as file:
    system_prompt = file.read()
system_prompt = system_prompt.replace("{", "{{").replace("}", "}}")

In [66]:
class StepResponse(BaseModel):
    title: str = Field(description="title of the step")
    content: str = Field(description="content of the step")
    confidence: str = Field(description="model confidence level for current step")
    next_action: str = Field(description="next action performed by the model")

In [71]:
parser = JsonOutputParser(pydantic_object=StepResponse)
# Define the prompt template
template = f"""{system_prompt}
Messages: {{messages}}
"""
# Create the prompt
prompt = ChatPromptTemplate.from_template(template)
print(prompt.input_variables)
model = OllamaLLM(model="codeup")
# Build the chain
chain = prompt | model | parser
user_message = "How many r are in strawberry. Input string is strawberry, ignore case sensitivity or any other edge cases."

['messages']


In [86]:
def generate_chain_of_thought(user_message, max_steps=10, confidence_threshold=99):
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_message),
        AIMessage(content="Understood. I will now create a detailed reasoning chain following the given instructions, starting with a thorough problem decomposition.")
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        try:
            step_data = chain.invoke({"messages": messages})
        except OutputParserException:
            step_data = {
                "title": "Raw Response",
                "content": steps[-1][1],
                "next_action": "continue"
            }
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        print(step_data)
        messages.append({"role": "assistant", "content": json.dumps(step_data)})
        print("Next reasoning step: ", step_data["title"])

        if step_data["next_action"].lower().strip() == "final_answer" or step_count > 10:
            break

        step_count += 1

    # Request final answer
    messages.append({
        "role": "user",
        "content": "Please provide the final answer based on your reasoning above.",
    })
    start_time = time.time()
    try:
        final_data = chain.invoke({"messages": messages})
    except OutputParserException:
        final_data = {
            "title": "Raw Response",
            "content": steps[-1][1],
            "next_action": "final_answer"
        }
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    # Add final answer to steps
    steps.append(("Final Answer", final_data["content"], thinking_time))

    # Yield final results
    return steps, total_thinking_time

In [89]:
user_message = "write hello world in java"
steps, total_time = generate_chain_of_thought(user_message)

{'title': 'Problem Decomposition', 'content': "To begin solving this problem, we need to decompose it into its core components. The given task is to write 'hello world' in Java. Let's break it down into the following steps:\n1. Identify the language and its syntax requirements\n2. Understand the basic structure of a Java program\n3. Write the 'hello world' message in Java\n4. Compile and run the program to ensure correct output\n\nNow, let me provide more details for each step...", 'confidence': 80, 'next_action': 'continue'}
Next reasoning step:  Problem Decomposition
{'title': 'Step 1: Identify Language and Syntax Requirements', 'content': "To begin with, we need to identify the language and its syntax requirements for writing 'hello world' in Java. The Java programming language is an object-oriented language that is widely used for developing desktop applications, mobile apps, web applications, and enterprise software. It is important to note that Java is a case-sensitive language, 

In [90]:
# Print the steps
print(f"{steps[-1][0]}\n{steps[-1][1]}\nTime Taken: {steps[-1][2]:.2f} seconds\n")
print(f"Total Thinking Time: {total_time:.2f} seconds")

Final Answer
To begin with, we need to identify the language and its syntax requirements for writing 'hello world' in Java. The Java programming language is an object-oriented language that is widely used for developing desktop applications, mobile apps, web applications, and enterprise software. It is important to note that Java is a case-sensitive language, and we need to follow the standard syntax and conventions when writing our code.
Time Taken: 24.66 seconds

Total Thinking Time: 108.59 seconds
